<b>Defining a Problem to Optimize</b>
<p>
Now we're going to put together a simple example of using a genetic algorithm in Python. We're going to optimize a very simple problem: trying to create a list of N numbers that equal X when summed together.
</p>
<br>
If we set N = 5 and X = 200, then these would all be appropriate solutions.
<br>
lst = [40,40,40,40,40]
<br>
lst = [50,50,50,25,25]
<br>
lst = [200,0,0,0,0]


<b>Ingredients of The Solution</b>

Each suggested solution for a genetic algorithm is referred to as an <b>individual</b>. In our current problem, each list of N numbers is an individual. 

In [29]:
from random import randint, random
from operator import add

In [2]:
def individual(length,minimum,maximum):
    return [randint(minimum, maximum) for x in range(length)]

In [3]:
individual(5,0,100)

[63, 79, 66, 50, 94]

In [4]:
individual(5,0,100)

[99, 24, 18, 41, 95]

A collection of individuals is known as population hence we create a population from multiple of these individuals

In [5]:
def population(count, length, minimum, maximum):
    return [individual(length,minimum,maximum) for x in range(count)]

In [6]:
population(3,5,0,100)

[[50, 56, 32, 60, 13], [79, 45, 8, 50, 89], [56, 68, 18, 36, 12]]

<p>Next we need a way to judge the how effective each solution is; to judge the fitness of each individual. Predictably enough, we call this the fitness function. For our problem, we want the fitness to be a function of the distance between the sum of an individuals numbers and the target number X.</p>
<br>
We can implement the fitness function as follows: 

In [11]:
def fitness(individual, target):
    print(individual)
    return abs(target-sum(individual))

In [12]:
x = individual(5,0, 100)
fitness(x,200)

[94, 57, 38, 31, 22]


42

Calculation as to how much diveation in general is possible for all the individuals in the population as shown here we take only 3 individuals in the population

In [24]:
def grade(population, target):
    temp = 0
    for i in population:
        temp = temp + abs(sum(i) - target)
    return temp/(len(population)*1.0)

In [25]:
x = population(3,5,0,100)
target = 200
grade(x,target)

34.666666666666664

<b>Evolution</b>

<p> Consider a population of sheep which are ruthlessly hunted by a pack of wolves. With each generation the weakest are eaten by the wolves, and then the strongest sheep reproduce and have children. Abstract those ideas a bit, and we can implement the evolution mechanism.

<p>1) <br>
For each generation we'll take a portion of the best performing individuals as judged by our fitness function. These high-performers will be the parents of the next generation.
</p>

<p>2)<br>
We'll also randomly select some lesser performing individuals to be parents, because we want to promote genetic diversity. Abandoning the metaphor, one of the dangers of optimization algorithms is getting stuck at a local maximum and consequently being unable to find the real maximum. By including some individuals who are not performing as well, we decrease our likelihood of getting stuck.
</p>
<p>3)
    <br>
Breed together parents to repopulate the population to its desired size (if you take the top 20 individuals in a population of 100, then you'd need to create 80 new children via breeding). In our case, breeding is pretty basic: take the first N/2 digits from the father and the last N/2 digits from the mother. 
</p>

In [28]:
father = [i for i in range(1,7)]
mother = [i for i in range(10,70,10)]
child = father[:3] + mother[3:]
print(child)

[1, 2, 3, 40, 50, 60]


<br>It's okay to have one parent breed multiple times, but one parent should never be both the father and mother of a child.
<br><br>
Merge together the parents and children to constitute the next generation's population.
<br><br>
Finally we mutate a small random portion of the population. What this means is to have a probability of randomly modifying each individual. 

In [36]:
pop = population(3,5,0,100)
print(pop)
chance_to_mutate = 0.1
for i in pop:
    if chance_to_mutate > random():
        place_to_modify = randint(0,len(i))
        i[place_to_modify] = randint(min(i),max(i))
        print(i)

[[56, 6, 0, 75, 7], [30, 43, 9, 42, 80], [60, 18, 39, 27, 22]]
[30, 43, 9, 47, 80]


This--just like taking individuals who are not performing particularly well--is to encourage genetic diversity, i.e. avoid getting stuck at local maxima.
<br><br>
Putting it all together, the code to evolve a generation can be implemented like this: 

In [40]:
def evolve(pop, target, retain=0.2, random_select=0.05, mutate=0.01):
    graded = [ (fitness(x, target), x) for x in pop]
    graded = [ x[1] for x in sorted(graded)]
    retain_length = int(len(graded)*retain)
    parents = graded[:retain_length]

    # randomly add other individuals to promote genetic diversity
    for individual in graded[retain_length:]:
        if random_select > random():
            parents.append(individual)
            
    # mutate some individuals
    for individual in parents:
        if mutate > random():
            pos_to_mutate = randint(0, len(individual)-1)
            individual[pos_to_mutate] = randint(min(individual), max(individual))
    
    # crossover parents to create children
    parents_length = len(parents)
    desired_length = len(pop) - parents_length
    children = []
    while len(children) < desired_length:
        male = randint(0, parents_length-1)
        female = randint(0, parents_length-1)
        if male != female:
            male = parents[male]
            female = parents[female]
            half = int(len(male) / 2)
            child = male[:half] + female[half:]
            children.append(child)

    parents.extend(children)
    return parents

In [44]:
target = 100
p_count = 100
i_length = 5
i_min = 0
i_max = 100
p = population(p_count, i_length, i_min, i_max)
fitness_history = [grade(p, target),]

In [45]:
for i in range(100):
    p = evolve(p,target)
    fitness_history.append(grade(p,target))

[58, 98, 24, 98, 49]
[60, 24, 3, 85, 10]
[27, 87, 86, 15, 13]
[38, 84, 65, 86, 53]
[76, 35, 67, 15, 4]
[2, 38, 59, 42, 48]
[91, 55, 74, 70, 83]
[87, 6, 1, 44, 94]
[35, 79, 39, 31, 26]
[28, 64, 57, 82, 65]
[32, 6, 60, 81, 12]
[3, 58, 5, 94, 81]
[34, 99, 41, 25, 59]
[84, 15, 62, 55, 50]
[42, 60, 51, 39, 72]
[18, 57, 14, 80, 82]
[8, 20, 40, 83, 72]
[12, 30, 49, 69, 48]
[92, 76, 74, 89, 29]
[50, 98, 8, 77, 73]
[47, 78, 59, 86, 75]
[48, 3, 15, 20, 11]
[82, 33, 43, 57, 28]
[79, 91, 73, 5, 45]
[3, 21, 89, 72, 72]
[72, 43, 7, 77, 7]
[20, 41, 25, 65, 2]
[62, 8, 98, 2, 68]
[20, 72, 18, 14, 32]
[91, 99, 98, 70, 42]
[17, 49, 83, 45, 58]
[27, 9, 37, 31, 60]
[76, 64, 4, 73, 36]
[44, 61, 31, 50, 86]
[12, 19, 5, 0, 42]
[12, 83, 80, 79, 25]
[81, 7, 73, 54, 6]
[24, 21, 10, 40, 40]
[87, 15, 91, 31, 20]
[57, 91, 10, 58, 24]
[34, 18, 55, 20, 9]
[52, 27, 26, 4, 75]
[82, 91, 25, 52, 97]
[26, 4, 75, 7, 9]
[78, 20, 24, 1, 77]
[80, 90, 100, 84, 33]
[8, 0, 13, 51, 87]
[65, 67, 24, 58, 84]
[7, 37, 26, 13, 32]
[16

[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]


[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]


[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]


[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]
[27, 9, 18, 14, 32]


In [46]:
print(sum([27, 9, 18, 14, 32]))

100
